In [74]:
from __future__ import print_function # So that this notebook becomes both Python 2 and Python 3 compatible
import numpy as np
import pandas as pd
from pprint import pprint
# !conda install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
from ipywidgets import interact, interact_manual
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import ipywidgets as widgets
from vega import VegaLite
import matplotlib.pyplot as plt

# def Vega(spec):
#     bundle = {}
#     bundle['application/vnd.vega.v4+json'] = spec
#     display(bundle, raw=True)

# def VegaLite(spec):
#     bundle = {}
#     bundle['application/vnd.vegalite.v2+json'] = spec
#     display(bundle, raw=True)

In [33]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum(np.square(x1-x2),axis=1))

def labels_equal(y1, y2):
    return (y1-y2)==0

def reshape_input(y_pred, y_true):
    xs, xt = y_pred[:,0], y_pred[:,1]
    ys, yt = y_true[:,0], y_true[:,1]

    X = np.concatenate([xs,xt])
    Y = np.concatenate([ys,yt])
    n = len(Y)
    return X, Y, n

def create_weight(Y, mode='pair'):
    n = len(Y)
    if mode=='pair':
        l = 1-labels_equal(Y[:n//2], Y[n//2:])
        zrs = np.zeros([int(n/2),int(n/2)])
        ind = np.diag(1-l) 
        ind_p = np.diag(l)
        W = np.array(np.concatenate([
            np.concatenate([zrs,ind], axis=1),
            np.concatenate([ind,zrs], axis=1)
        ]))
        W_p = np.array(np.concatenate([
            np.concatenate([zrs,ind_p], axis=1),
            np.concatenate([ind_p,zrs], axis=1)
        ]))
    elif mode in ['all', 'across']:
        YY = np.broadcast_to(Y,shape=(n,n))
        W = (YY == YY.T)-np.eye(n)
        W_p = 1-W-np.eye(n)
        if mode == 'across':
            mask = np.repeat(np.repeat(np.array([[0,1],[1,0]]),n//2,axis=0),n//2,axis=1)
            W = W * mask
            W_p = W_p * mask
    else:
        raise ValueError("mode must be one of ['pair', 'all', 'across']") 
    return W, W_p



In [45]:
def csa(y_pred, y_true, margin = 1):
    X, Y, n = reshape_input(y_pred, y_true)
    
    xs, xt = X[:n//2], X[n//2:]
    ys, yt = Y[:n//2], Y[n//2:]

    dist = euclidean_distance(xs, xt)
    l = 1-labels_equal(ys, yt) # 0 for equals

    losses = (1-l) * 0.5*np.square(dist) + (l) * 0.5*np.square(np.maximum(margin - dist, 0))
    return losses

In [119]:
def graph_csa(y_pred, y_true, margin = 1):
    X, Y, n = reshape_input(y_pred, y_true)
    W, W_p = create_weight(Y,mode='pair')
    l = 1-labels_equal(Y[:n//2], Y[n//2:])

    # Degree matrix
    D = np.diag(np.sum(W,axis=0))
    D_p = np.diag(np.sum(W_p,axis=0))

    # Graph Laplacian
    L = D - W
    B = D_p - W_p

    XLX = X.T @ L @ X 
    XBX = X.T @ B @ X 
    XBXeps = np.maximum(0,np.square(margin)+XBX-2*np.sqrt(XBX)*margin) *(XBX != 0)

    L_s = 0.5*np.linalg.norm(XLX, ord='fro')
    L_d = 0.5*np.linalg.norm(XBXeps, ord='fro')
#     L_s = 0.5*np.trace(XLX)
#     L_d = 0.5*np.trace(XBXeps)

    L_csa = (1-l)*L_s + l*L_d
    return L_csa

In [107]:
def homebrew(y_pred, y_true):
    X, Y, n = reshape_input(y_pred, y_true)
    W, W_p = create_weight(Y,mode='all')

    # epsilon thresholding or nearest neighbor threshold?
    # TODO

    # Degree matrix
    D = np.diag(np.sum(W,axis=0))
    D_p = np.diag(np.sum(W_p,axis=0))

    # Graph Laplacian
    L = D - W
    B = D_p - W_p

    XLX = X.T @ L @ X 
    XBX = X.T @ B @ X 
    # The diagonal of XLX and XBX are the sum of squared differences along the dimension
    
    print('XLX = \n',XLX)
    print('XLX = \n',XBX)

    L_hb = np.trace(XLX) / np.trace(XBX)
    return L_hb

# Experiments

In [91]:
def scatter(XY):
    return VegaLite({
        "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
        "description": "A simple bar chart with embedded data.",
        "width": 300,
        "height": 300,
        "data": {
            "values": [{'x1':xy[0],'x2':xy[1],'y':xy[2]} for xy in list(XY)],
            "on": [
                {"trigger": "whichPoint", "modify": "whichPoint", "values": "{x1: newPointPosition.u, x2:newPointPosition.v}"}
            ],
        },
        "mark": {
            "type": "circle",
            "size":50,
        },
        "encoding": {
            "x": {
                "field": "x1",
                "type": "quantitative",
                "scale": {"domain": [0,5]},
            },
            "y": {
                "field": "x2",
                "type": "quantitative",
                "scale": {"domain": [0,5]}
            },
            "color": {"field": "y", "type": "nominal"},
        }
    })

def scatter2(X,Y):
    plt.scatter(X[:,0], X[:,1], c=Y, alpha=0.5)
    plt.ylim((-3, 3)) 
    plt.xlim((-2, 5)) 
    plt.show()

In [120]:
@interact
def sliders(x_in=1.0, y_in=1.0, x_out=2.0, y_out=1.0):
    y_true = np.array([[1,1],
                       [2,1]])
    y_pred = 1*np.array([[[2,2],[1,2]],
                         [[x_out,y_out],[x_in,y_in]]])
    X, Y, n = reshape_input(y_pred, y_true)
#     XY = np.concatenate([X.T, np.reshape(Y,(4,1)).T]).T
    print({
        'csa':       "{0:.2f}".format(np.sum(csa(y_pred, y_true))),
#         'csa_graph': "{0:.2f}".format(np.sum(graph_csa(y_pred,y_true))),
        'homebrew':  "{0:.2f}".format(homebrew(y_pred, y_true))
    })
    scatter2(X,Y)
    

interactive(children=(FloatSlider(value=1.0, description='x_in', max=3.0, min=-1.0), FloatSlider(value=1.0, de…

In [123]:
θϕLϕθ = 42

In [124]:
θϕLϕθ

42

In [126]:
2 != 1

True